In [2]:
import cv2
import mediapipe as mp

# Inicializa MediaPipe
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Configura la cámara
cap = cv2.VideoCapture(0)

# Verifica si la cámara está disponible
if not cap.isOpened():
    print("Error: No se puede acceder a la cámara.")
    exit()

# Inicia el modelo de detección de pose
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("No se pudo leer el cuadro de la cámara.")
            break

        
        # Convierte la imagen de BGR a RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Procesa la imagen para detectar poses
        results = pose.process(rgb_frame)

        # Dibuja las anotaciones de pose en la imagen original
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)
            )

        # Muestra la imagen procesada
        cv2.imshow('Skeleton Detection', frame)

        # Salir con la tecla 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Libera los recursos
cap.release()
cv2.destroyAllWindows()


In [4]:
import cv2
import mediapipe as mp
import time

# Inicializa los módulos de Mediapipe
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Configura la cámara y el video
cap = cv2.VideoCapture(0)  # Cámara en vivo
video_cap = cv2.VideoCapture('dance_video.mp4')  # Video del archivo

# Variable para medir el tiempo
start_time = time.time()

# Inicia el modelo de detección de pose
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while True:
        # Captura cuadro del video
        ret_vid, frame_vid = video_cap.read()
        if not ret_vid:
            print("Se alcanzó el final del video.")
            video_cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Reinicia el video
            ret_vid, frame_vid = video_cap.read()

        frame_vid = cv2.resize(frame_vid, None, fx=0.5, fy=0.5)
        # Procesa el video
        rgb_frame_vid = cv2.cvtColor(frame_vid, cv2.COLOR_BGR2RGB)
        results_vid = pose.process(rgb_frame_vid)

        if results_vid.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame_vid, results_vid.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2, circle_radius=2)
            )

        # Captura los puntos cada 30 segundos
        current_time = time.time()
        if current_time - start_time >= 30:

            # Imprime los puntos del video en la terminal
            if results_vid.pose_landmarks:
                print("Puntos detectados (Video):")
                for id, landmark in enumerate(results_vid.pose_landmarks.landmark):
                    print(f"Punto {id}: (x: {landmark.x}, y: {landmark.y}, z: {landmark.z}, visibilidad: {landmark.visibility})")
                print("\n---\n")

            # Reinicia el contador de tiempo
            start_time = current_time

        # Muestra las imágenes procesadas
        cv2.imshow('Skeleton Detection (Video)', frame_vid)

        # Termina el bucle si se presiona la tecla 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Libera los recursos
cap.release()
video_cap.release()
cv2.destroyAllWindows()

In [5]:
import cv2
import mediapipe as mp
import numpy as np
import json

# Inicializa los módulos de Mediapipe
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Configuración del video de referencia
dance_video = 'dance_video.mp4'

# Función para extraer puntos de referencia y guardarlos en un archivo JSON
def extraer_y_guardar_puntos(video_path, output_file):
    puntos_video_referencia = []

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        video_cap = cv2.VideoCapture(video_path)
        while True:
            ret_vid, frame_vid = video_cap.read()
            if not ret_vid:
                break

            rgb_frame_vid = cv2.cvtColor(frame_vid, cv2.COLOR_BGR2RGB)
            results_vid = pose.process(rgb_frame_vid)

            if results_vid.pose_landmarks:
                puntos = []
                for landmark in results_vid.pose_landmarks.landmark:
                    puntos.append({
                        "x": landmark.x,
                        "y": landmark.y,
                        "z": landmark.z,
                        "visibility": landmark.visibility
                    })
                puntos_video_referencia.append(puntos)

        video_cap.release()

    # Guarda los puntos en un archivo JSON
    with open(output_file, 'w') as f:
        json.dump(puntos_video_referencia, f)

    print(f"Puntos de referencia guardados en {output_file}")

# Función para calcular la distancia relativa entre puntos clave
def calcular_distancia_escala(puntos):
    """
    Calcula una medida de escala basada en la distancia entre los hombros
    (puntos 11 y 12 en Mediapipe Pose).
    """
    hombro_izquierdo = puntos[11]
    hombro_derecho = puntos[12]
    return np.linalg.norm(np.array([hombro_izquierdo["x"], hombro_izquierdo["y"]]) -
                          np.array([hombro_derecho["x"], hombro_derecho["y"]]))

# Código para jugar comparando los puntos guardados
def jugar_con_puntos(json_file, video_path):
    # Carga los puntos del archivo JSON
    with open(json_file, 'r') as f:
        puntos_video_referencia = json.load(f)

    cap = cv2.VideoCapture(0)  # Cámara en vivo
    video_cap = cv2.VideoCapture(video_path)  # Video de referencia

    # Obtén la tasa de fotogramas por segundo (FPS) del video de referencia
    fps_video = video_cap.get(cv2.CAP_PROP_FPS)
    frame_duration = 1 / fps_video  # Duración de cada cuadro en segundos

    def calcular_similitud(puntos1, puntos2, escala1, escala2):
        """
        Calcula la similitud entre dos poses ajustando las coordenadas por sus escalas relativas.
        """
        if not puntos1 or not puntos2 or len(puntos1) != len(puntos2):
            return 0

        distancias = []
        for p1, p2 in zip(puntos1, puntos2):
            p1_escalado = np.array([p1["x"] / escala1, p1["y"] / escala1, p1["z"] / escala1])
            p2_escalado = np.array([p2["x"] / escala2, p2["y"] / escala2, p2["z"] / escala2])
            distancias.append(np.linalg.norm(p1_escalado - p2_escalado))

        return np.mean(distancias)

    current_time = 0
    while True:
        # Leer cuadro del video de referencia
        ret_vid, frame_vid = video_cap.read()
        if not ret_vid:
            video_cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Reinicia el video
            ret_vid, frame_vid = video_cap.read()
            current_time = 0

        # Calcular el índice del JSON basado en el tiempo transcurrido
        frame_idx = int(current_time * fps_video)
        if frame_idx < len(puntos_video_referencia):
            puntos_referencia = puntos_video_referencia[frame_idx]

            # Dibujar puntos y conexiones en el video de referencia
            altura, ancho, _ = frame_vid.shape
            for conexion in mp_pose.POSE_CONNECTIONS:
                inicio = puntos_referencia[conexion[0]]
                fin = puntos_referencia[conexion[1]]
                inicio_px = (int(inicio["x"] * ancho), int(inicio["y"] * altura))
                fin_px = (int(fin["x"] * ancho), int(fin["y"] * altura))
                cv2.line(frame_vid, inicio_px, fin_px, (255, 0, 0), 2)

            for punto in puntos_referencia:
                px = (int(punto["x"] * ancho), int(punto["y"] * altura))
                cv2.circle(frame_vid, px, 5, (0, 255, 0), -1)

        # Leer cuadro de la cámara en vivo
        ret_cam, frame_cam = cap.read()
        if not ret_cam:
            print("No se puede acceder a la cámara.")
            break

        rgb_frame_cam = cv2.cvtColor(frame_cam, cv2.COLOR_BGR2RGB)
        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
            results_cam = pose.process(rgb_frame_cam)

            if results_cam.pose_landmarks:
                puntos_usuario = [
                    {"x": lm.x, "y": lm.y, "z": lm.z, "visibility": lm.visibility}
                    for lm in results_cam.pose_landmarks.landmark
                ]

                if frame_idx < len(puntos_video_referencia):
                    escala_referencia = calcular_distancia_escala(puntos_referencia)
                    escala_usuario = calcular_distancia_escala(puntos_usuario)

                    similitud = calcular_similitud(puntos_referencia, puntos_usuario, escala_referencia, escala_usuario)

                    if similitud < 0.1:
                        puntuacion = 100
                    elif similitud < 0.2:
                        puntuacion = 50
                    else:
                        puntuacion = 0

                    cv2.putText(frame_cam, f'Puntuacion: {puntuacion}', (10, 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            if results_cam.pose_landmarks:
                mp_drawing.draw_landmarks(
                    frame_cam, results_cam.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(255, 0, 255), thickness=2, circle_radius=2)
                )

        # Mostrar ambos cuadros
        combinado = np.hstack((cv2.resize(frame_vid, (640, 480)), cv2.resize(frame_cam, (640, 480))))
        cv2.imshow('Comparacion: Video vs Usuario', combinado)

        # Incrementar el tiempo actual según la duración de cada cuadro
        current_time += frame_duration

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    video_cap.release()
    cv2.destroyAllWindows()

# Ejemplo de uso
if __name__ == "__main__":
    modo = input("Seleccione modo: 'extraer' para guardar puntos o 'jugar' para comparar: ")

    if modo == 'extraer':
        extraer_y_guardar_puntos(dance_video, 'puntos_referencia.json')
    elif modo == 'jugar':
        jugar_con_puntos('puntos_referencia.json', dance_video)


In [1]:
import cv2
import mediapipe as mp
import numpy as np
import json

# Inicializa los módulos de Mediapipe
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Configuración del video de referencia
dance_video = 'dance_video.mp4'



# Función para calcular la distancia relativa entre puntos clave
def calcular_distancia_escala(puntos):
    """
    Calcula una medida de escala basada en la distancia entre los hombros
    (puntos 11 y 12 en Mediapipe Pose).
    """
    hombro_izquierdo = puntos[11]
    hombro_derecho = puntos[12]
    return np.linalg.norm(np.array([hombro_izquierdo["x"], hombro_izquierdo["y"]]) -
                          np.array([hombro_derecho["x"], hombro_derecho["y"]]))

# Código para jugar comparando los puntos guardados

def jugar_con_puntos(json_file, video_path):
    # Carga los puntos del archivo JSON
    with open(json_file, 'r') as f:
        puntos_video_referencia = json.load(f)

    #cap = cv2.VideoCapture(0)  # Cámara en vivo
    video_cap = cv2.VideoCapture(video_path)  # Video de referencia

    # def calcular_similitud(puntos1, puntos2, escala1, escala2):
    #     """
    #     Calcula la similitud entre dos poses ajustando las coordenadas por sus escalas relativas.
    #     """
    #     if not puntos1 or not puntos2 or len(puntos1) != len(puntos2):
    #         return 0

    #     distancias = []
    #     for p1, p2 in zip(puntos1, puntos2):
    #         p1_escalado = np.array([p1["x"] / escala1, p1["y"] / escala1, p1["z"] / escala1])
    #         p2_escalado = np.array([p2["x"] / escala2, p2["y"] / escala2, p2["z"] / escala2])
    #         distancias.append(np.linalg.norm(p1_escalado - p2_escalado))

    #     return np.mean(distancias)

    frame_idx = 0

    while True:
        # Leer cuadro del video de referencia
        if frame_idx < len(puntos_video_referencia):
            referencia = puntos_video_referencia[frame_idx]
            puntos_referencia = referencia["puntos"]
            frame_index = referencia["frame"]

            # Posicionar el video en el frame correspondiente
            video_cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
            ret_vid, frame_vid = video_cap.read()

            if not ret_vid:
                break

            # Dibujar puntos y conexiones en el video de referencia
            altura, ancho, _ = frame_vid.shape
            for conexion in mp_pose.POSE_CONNECTIONS:
                inicio = puntos_referencia[conexion[0]]
                fin = puntos_referencia[conexion[1]]
                inicio_px = (int(inicio["x"] * ancho), int(inicio["y"] * altura))
                fin_px = (int(fin["x"] * ancho), int(fin["y"] * altura))
                cv2.line(frame_vid, inicio_px, fin_px, (255, 0, 0), 2)

            for punto in puntos_referencia:
                px = (int(punto["x"] * ancho), int(punto["y"] * altura))
                cv2.circle(frame_vid, px, 5, (0, 255, 0), -1)

        # Leer cuadro de la cámara en vivo
        # ret_cam, frame_cam = cap.read()
        # if not ret_cam:
        #     print("No se puede acceder a la cámara.")
        #     break

        # frame_cam = cv2.resize(frame_cam, None, fx=0.5, fy=0.5)
        # rgb_frame_cam = cv2.cvtColor(frame_cam, cv2.COLOR_BGR2RGB)
        # with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        #     results_cam = pose.process(rgb_frame_cam)

        #     if results_cam.pose_landmarks:
        #         puntos_usuario = [
        #             {"x": lm.x, "y": lm.y, "z": lm.z, "visibility": lm.visibility}
        #             for lm in results_cam.pose_landmarks.landmark
        #         ]

                # if frame_idx < len(puntos_video_referencia):
                #     escala_referencia = calcular_distancia_escala(puntos_referencia)
                #     escala_usuario = calcular_distancia_escala(puntos_usuario)

                #     similitud = calcular_similitud(puntos_referencia, puntos_usuario, escala_referencia, escala_usuario)

                #     if similitud < 0.1:
                #         puntuacion = 100
                #     elif similitud < 0.2:
                #         puntuacion = 50
                #     else:
                #         puntuacion = 0

                #     cv2.putText(frame_cam, f'Puntuacion: {puntuacion}', (10, 50),
                #                 cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            # if results_cam.pose_landmarks:
            #     mp_drawing.draw_landmarks(
            #         frame_cam, results_cam.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            #         mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2, circle_radius=2),
            #         mp_drawing.DrawingSpec(color=(255, 0, 255), thickness=2, circle_radius=2)
            #     )

        # Mostrar ambos cuadros
        # combinado = np.hstack((cv2.resize(frame_vid, (640, 480)), cv2.resize(frame_cam, (640, 480))))

        cv2.imshow('Comparacion: Video vs Usuario', frame_vid)

        frame_idx += 1

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    video_cap.release()
    cv2.destroyAllWindows()

# Ejemplo de uso
if __name__ == "__main__":
    modo = input("Seleccione modo: 'extraer' para guardar puntos o 'jugar' para comparar: ")

    if modo == 'extraer':
        extraer_y_guardar_puntos(dance_video, 'puntos_referencia.json')
    elif modo == 'jugar':
        jugar_con_puntos('puntos_referencia.json', dance_video)


NameError: name 'cap' is not defined

: 

In [1]:
import tensorflow as tf
print("GPU available:", tf.config.list_physical_devices('GPU'))

GPU available: []


In [ ]:
import cv2
import pygame
import mediapipe as mp
import numpy as np
import json


# Inicializa los módulos de Mediapipe
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Configuración del video de referencia
dance_video = 'dance_video.mp4'


# Función para calcular la distancia relativa entre puntos clave
def calcular_distancia_escala(puntos):
    """
    Calcula una medida de escala basada en la distancia entre los hombros
    (puntos 11 y 12 en Mediapipe Pose).
    """
    hombro_izquierdo = puntos[11]
    hombro_derecho = puntos[12]
    return np.linalg.norm(np.array([hombro_izquierdo["x"], hombro_izquierdo["y"]]) -
                          np.array([hombro_derecho["x"], hombro_derecho["y"]]))


def calcular_similitud(puntos1, puntos2, escala1, escala2):
        """
        Calcula la similitud entre dos poses ajustando las coordenadas por sus escalas relativas.
        """
        if not puntos1 or not puntos2 or len(puntos1) != len(puntos2):
            return 0

        distancias = []
        for p1, p2 in zip(puntos1, puntos2):
            p1_escalado = np.array([p1["x"] / escala1, p1["y"] / escala1, p1["z"] / escala1])
            p2_escalado = np.array([p2["x"] / escala2, p2["y"] / escala2, p2["z"] / escala2])
            distancias.append(np.linalg.norm(p1_escalado - p2_escalado))

        return np.mean(distancias)



def obtener_puntos_por_frame(json_data, frame_buscado):
    # Buscar el frame en el JSON
    for entrada in json_data:
        if entrada["frame"] == frame_buscado:
            return entrada["puntos"]
    return None  # Si no se encuentra el frame



def puntuacion(json_file, frame_buscado,results_cam, frame_cam):
    puntos_referencia = obtener_puntos_por_frame (json_file, frame_buscado)
    if(puntos_referencia):
        puntos_usuario = [
                    {"x": lm.x, "y": lm.y, "z": lm.z, "visibility": lm.visibility}
                    for lm in results_cam.pose_landmarks.landmark
                ]
        
        escala_referencia = calcular_distancia_escala(puntos_referencia)
        escala_usuario = calcular_distancia_escala(puntos_usuario)

        similitud = calcular_similitud(puntos_referencia, puntos_usuario, escala_referencia, escala_usuario)

        print(similitud)
    
        if similitud < 5:
            puntuacion = 100
        elif similitud < 10:
            puntuacion = 50
        else:
            puntuacion = 0

        return(puntuacion)

        
        

def jugar_con_puntos(json_file, video_path):

    with open(json_file, 'r') as f:
        puntos_video_referencia = json.load(f)

    # Inicializar Pygame
    pygame.init()

    # Cargar y reproducir el archivo de audio
    audio_file = 'audio_extraido.mp3'  # Reemplaza con la ruta a tu archivo de audio
    pygame.mixer.music.load(audio_file)
    pygame.mixer.music.play()

    # Cargar el video usando OpenCV
    cap_video = cv2.VideoCapture(video_path)

    # Verifica si se pudo abrir el archivo de video
    if not cap_video.isOpened():
        print("Error: No se pudo abrir el archivo de video.")
        exit()

    # Obtener las dimensiones del video (ancho y alto)
    frame_width = int(cap_video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap_video.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Configuración de la ventana de Pygame
    screen_width = frame_width * 2  # Pantalla con el doble de ancho (video + cámara)
    screen_height = frame_height  # Mantener la altura igual
    screen = pygame.display.set_mode((screen_width, screen_height))  # Tamaño de la ventana

    # Obtener la tasa de fotogramas del video (FPS)
    video_frame_rate = cap_video.get(cv2.CAP_PROP_FPS)

    # Inicialización de variables
    last_frame_index = -1

    # Configura la cámara
    cap_camera = cv2.VideoCapture(0)

    # Verifica si la cámara está disponible
    if not cap_camera.isOpened():
        print("Error: No se puede acceder a la cámara.")
        exit()

    score=0

    # Inicia el modelo de detección de pose
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

    
        while cap_video.isOpened() and cap_camera.isOpened():
            # Obtiene el tiempo actual de la música en segundos
            current_time = pygame.mixer.music.get_pos() / 1_000  # en segundos
            current_frame_index = int(current_time * video_frame_rate)  # Índice del fotograma correspondiente

            # Si el índice del fotograma ha cambiado, leemos y mostramos el nuevo fotograma
            if current_frame_index != last_frame_index:
                # Rewind a la posición correcta en el video
                cap_video.set(cv2.CAP_PROP_POS_FRAMES, current_frame_index)

                # Lee el fotograma del video
                ret_video, frame_video = cap_video.read()
                if not ret_video:
                    break  # Si no hay más frames, salimos del bucle

                # Convertir la imagen de BGR (OpenCV) a RGB (Pygame)
                frame_rgb = cv2.cvtColor(frame_video, cv2.COLOR_BGR2RGB)
                current_image = pygame.image.frombuffer(frame_rgb.tobytes(), frame_rgb.shape[1::-1], "RGB")

                last_frame_index = current_frame_index


            # Mostrar el fotograma del video a la izquierda
            screen.blit(current_image, (0, 0))

            # Leer un fotograma de la cámara
            ret_camera, frame_camera = cap_camera.read()
            if not ret_camera:
                print("No se pudo leer el cuadro de la cámara.")
                break

            # Procesa la imagen para detectar poses
            # results = pose.process(rgb_frame_camera)

            # # Dibuja las anotaciones de pose en la imagen original de la cámara
            # if results.pose_landmarks:
            #     mp_drawing.draw_landmarks(
            #         frame_camera, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            #         mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            #         mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)
            #     )
            

                # # Dibujar puntos y conexiones en el video de referencia
                # altura, ancho, _ = frame_camera.shape
                # for conexion in mp_pose.POSE_CONNECTIONS:
                #     inicio = puntos_referencia[conexion[0]]
                #     fin = puntos_referencia[conexion[1]]
                #     inicio_px = (int(inicio["x"] * ancho), int(inicio["y"] * altura))
                #     fin_px = (int(fin["x"] * ancho), int(fin["y"] * altura))
                #     cv2.line(frame_camera, inicio_px, fin_px, (255, 0, 0), 2)

                # for punto in puntos_referencia:
                #     px = (int(punto["x"] * ancho), int(punto["y"] * altura))
                #     cv2.circle(frame_camera, px, 5, (0, 255, 0), -1)

            frame_camera = cv2.flip(frame_camera, 1)
            rgb_frame_cam = cv2.cvtColor(frame_camera, cv2.COLOR_BGR2RGB)
            results_cam = pose.process(rgb_frame_cam)


            if results_cam.pose_landmarks:
                if current_frame_index % 15 == 0:
                    score += puntuacion(puntos_video_referencia,current_frame_index,results_cam, frame_camera)
                mp_drawing.draw_landmarks(
                    frame_camera, results_cam.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(255, 0, 255), thickness=2, circle_radius=2)
                )

            cv2.putText(frame_camera, f'Puntuacion: {score}', (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
            # Convertir la imagen de la cámara a RGB para Pygame
            frame_camera_rgb = cv2.cvtColor(frame_camera, cv2.COLOR_BGR2RGB)
            frame_camera_surface = pygame.image.frombuffer(frame_camera_rgb.tobytes(), frame_camera_rgb.shape[1::-1], "RGB")

            # Mostrar la imagen de la cámara a la derecha
            screen.blit(frame_camera_surface, (frame_width, 0))

            pygame.display.flip()

            # Comprobar eventos, como cerrar la ventana
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    cap_video.release()
                    cap_camera.release()
                    pygame.quit()
                    exit()

    # Cerrar todo después de la reproducción
    cap_video.release()
    cap_camera.release()
    pygame.quit()

# Función para extraer puntos de referencia y guardarlos en un archivo JSON
def extraer_y_guardar_puntos(video_path, output_file):
    puntos_video_referencia = []

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        video_cap = cv2.VideoCapture(video_path)
        while True:
            ret_vid, frame_vid = video_cap.read()
            if not ret_vid:
                break

            # Obtén el índice del frame actual
            frame_index = int(video_cap.get(cv2.CAP_PROP_POS_FRAMES))

            rgb_frame_vid = cv2.cvtColor(frame_vid, cv2.COLOR_BGR2RGB)
            results_vid = pose.process(rgb_frame_vid)

            if results_vid.pose_landmarks:
                puntos = []
                for landmark in results_vid.pose_landmarks.landmark:
                    puntos.append({
                        "x": landmark.x,
                        "y": landmark.y,
                        "z": landmark.z,
                        "visibility": landmark.visibility
                    })

                # Agrega los puntos junto con el índice del frame al resultado
                puntos_video_referencia.append({
                    "frame": frame_index,
                    "puntos": puntos
                })

        video_cap.release()

    # Guarda los puntos en un archivo JSON
    with open(output_file, 'w') as f:
        json.dump(puntos_video_referencia, f)

    print(f"Puntos de referencia guardados en {output_file}")
    
if __name__ == "__main__":
    modo = input("Seleccione modo: 'extraer' para guardar puntos o 'jugar' para comparar: ")

    if modo == 'extraer':
        extraer_y_guardar_puntos(dance_video, 'puntos_referencia.json')
    elif modo == 'jugar':
        jugar_con_puntos('puntos_referencia.json', dance_video)
        

24.03846546651225
29.13448370694829
8.457150036738447
5.787539773231634
5.89521938704435
5.970348683331194
5.384230527295914
4.498148092636044
4.768854737994984
5.015178541974257
5.589208446900982
5.984069798576098
5.194977309336598
4.353297254685842
5.7779338298319995


: 

In [1]:
import cv2
import pygame
import mediapipe as mp
import numpy as np
import json
from queue import Queue
from threading import Thread

# Inicializa los módulos de Mediapipe
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils


# Configuración del video de referencia
dance_video = 'dance_video.mp4'

# Configuración
process_every_n_frames = 15

def calcular_distancia_escala(puntos):
    hombro_izquierdo = puntos[11]
    hombro_derecho = puntos[12]
    return np.linalg.norm(np.array([hombro_izquierdo["x"], hombro_izquierdo["y"]]) -
                          np.array([hombro_derecho["x"], hombro_derecho["y"]]))

def calcular_similitud(puntos1, puntos2, escala1, escala2):
    if not puntos1 or not puntos2 or len(puntos1) != len(puntos2):
        return 0

    distancias = []
    for p1, p2 in zip(puntos1, puntos2):
        p1_escalado = np.array([p1["x"] / escala1, p1["y"] / escala1, p1["z"] / escala1])
        p2_escalado = np.array([p2["x"] / escala2, p2["y"] / escala2, p2["z"] / escala2])
        distancias.append(np.linalg.norm(p1_escalado - p2_escalado))

    return np.mean(distancias)

def obtener_puntos_por_frame(json_data, frame_buscado):
    for entrada in json_data:
        if entrada["frame"] == frame_buscado:
            return entrada["puntos"]
    return None

def puntuacion(json_file, frame_buscado, results_cam):
    puntos_referencia = obtener_puntos_por_frame(json_file, frame_buscado)
    if puntos_referencia:
        puntos_usuario = [
            {"x": lm.x, "y": lm.y, "z": lm.z, "visibility": lm.visibility}
            for lm in results_cam.pose_landmarks.landmark
        ]

        escala_referencia = calcular_distancia_escala(puntos_referencia)
        escala_usuario = calcular_distancia_escala(puntos_usuario)

        similitud = calcular_similitud(puntos_referencia, puntos_usuario, escala_referencia, escala_usuario)

        print(similitud)

        if similitud < 2:
            return 100
        elif similitud < 5:
            return 50
        else:
            return 0
    return 0

def process_video(cap_video, video_queue, video_frame_rate):
    while cap_video.isOpened():
        current_time = pygame.mixer.music.get_pos() / 1_000
        current_frame_index = int(current_time * video_frame_rate)

        cap_video.set(cv2.CAP_PROP_POS_FRAMES, current_frame_index)
        ret_video, frame_video = cap_video.read()
        if not ret_video:
            break

        frame_rgb = cv2.cvtColor(frame_video, cv2.COLOR_BGR2RGB)
        video_queue.put(frame_rgb)

    cap_video.release()

def process_camera(cap_camera, camera_queue, json_file, video_frame_rate):
    frame_count = 0
    score = 0
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap_camera.isOpened():
            ret_camera, frame_camera = cap_camera.read()
            if not ret_camera:
                break

            frame_camera = cv2.flip(frame_camera, 1)
            rgb_frame_cam = cv2.cvtColor(frame_camera, cv2.COLOR_BGR2RGB)

            frame_count += 1
            if frame_count % process_every_n_frames == 0:
                results_cam = pose.process(rgb_frame_cam)
                if results_cam.pose_landmarks:
                    current_time = pygame.mixer.music.get_pos() / 1_000
                    current_frame_index = int(current_time * video_frame_rate)
                    score += puntuacion(json_file, current_frame_index, results_cam)

                    mp_drawing.draw_landmarks(
                        frame_camera, results_cam.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2, circle_radius=2),
                        mp_drawing.DrawingSpec(color=(255, 0, 255), thickness=2, circle_radius=2)
                    )

            cv2.putText(frame_camera, f'Puntuacion: {score}', (10, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            camera_queue.put(frame_camera)

    cap_camera.release()

def jugar_con_puntos(json_file, video_path):
    with open(json_file, 'r') as f:
        puntos_video_referencia = json.load(f)

    pygame.init()
    audio_file = 'audio_extraido.mp3'
    pygame.mixer.music.load(audio_file)
    pygame.mixer.music.play()

    cap_video = cv2.VideoCapture(video_path)
    cap_camera = cv2.VideoCapture(0)

    if not cap_video.isOpened() or not cap_camera.isOpened():
        print("Error: No se pudo abrir el video o la cámara.")
        return

    frame_width = int(cap_video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    video_frame_rate = cap_video.get(cv2.CAP_PROP_FPS)

    screen_width = frame_width * 2
    screen_height = frame_height
    screen = pygame.display.set_mode((screen_width, screen_height))

    video_queue = Queue()
    camera_queue = Queue()

    video_thread = Thread(target=process_video, args=(cap_video, video_queue, video_frame_rate))
    camera_thread = Thread(target=process_camera, args=(cap_camera, camera_queue, puntos_video_referencia, video_frame_rate))

    video_thread.start()
    camera_thread.start()

    while video_thread.is_alive() or camera_thread.is_alive():
        if not video_queue.empty():
            frame_rgb = video_queue.get()
            current_image = pygame.image.frombuffer(frame_rgb.tobytes(), frame_rgb.shape[1::-1], "RGB")
            screen.blit(current_image, (0, 0))

        if not camera_queue.empty():
            frame_camera = camera_queue.get()
            frame_camera_rgb = cv2.cvtColor(frame_camera, cv2.COLOR_BGR2RGB)
            frame_camera_surface = pygame.image.frombuffer(frame_camera_rgb.tobytes(), frame_camera_rgb.shape[1::-1], "RGB")
            screen.blit(frame_camera_surface, (frame_width, 0))

        pygame.display.flip()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                video_thread.join()
                camera_thread.join()
                cap_video.release()
                cap_camera.release()
                pygame.quit()
                return

    video_thread.join()
    camera_thread.join()
    cap_video.release()
    cap_camera.release()
    pygame.quit()

if __name__ == "__main__":
    modo = input("Seleccione modo: 'extraer' para guardar puntos o 'jugar' para comparar: ")

    if modo == 'extraer':
        extraer_y_guardar_puntos(dance_video, 'puntos_referencia.json')
    elif modo == 'jugar':
        jugar_con_puntos('puntos_referencia.json', dance_video)
        

pygame 2.6.1 (SDL 2.28.4, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
5.533068038962008
6.367608347834825
20.885193980870753
9.296929396072782
8.62524713220261
42.86169065263305
27.906333946983725
7.153969500210845
8.067364188878496
110.21935719605897
38.23720911571398
6.582141244279704
21.699594987259616
62.82805396826064
40.88977732830045
7.032170502806466
22.218139951000264
61.53372397735613
8.980719586486218
13.260115104847726
3.739607167184584
3.2214150432950888
4.439356544133818
3.90633826991252
3.7558273596341647
2.716533441268541
4.2955027706112405
2.8849451586778154
3.711706134186792
4.183505686051071
4.341374347522366
4.595540911738387
3.014187731498405
3.5499272283046746
4.226436845072589
4.726238466106987
3.5733135792707684
4.055590563949639
4.269661064440211
5.110676673712902
4.287987373889075
4.558664540184836
4.564303509362666
3.4821667612760465
3.308771549441922
4.087936055242248
4.320761742241466
3.614102204177619
3.8328500457

: 

In [6]:
import cv2
import pygame
import mediapipe as mp
import numpy as np
import json
from PIL import Image, ImageSequence
from queue import Queue
from threading import Thread

# Inicializa los módulos de Mediapipe
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils


# Configuración del video de referencia
dance_video = 'dance_video.mp4'

# Configuración
process_every_n_frames = 30

stop_threads = False  # Bandera para detener los hilos

# Crear un índice para cambiar entre los frames del GIF
gif_index = 0
# Convertir el GIF en una lista de frames de imágenes
gif_frames = []

def overlay_image(frame, overlay, x, y):
    """
    Superpone una imagen sobre un frame en las coordenadas (x, y).
    """
    overlay_height, overlay_width = overlay.shape[:2]
    alpha_overlay = None

    # Si la imagen tiene canal alfa, úsalo
    if overlay.shape[2] == 4:
        alpha_overlay = overlay[:, :, 3] / 255.0  # Normaliza el canal alfa
        overlay = overlay[:, :, :3]  # Elimina el canal alfa para la mezcla

    # Seleccionar la región de interés (ROI) en el frame
    roi = frame[y:y+overlay_height, x:x+overlay_width]

    # Combinación usando el canal alfa (si existe)
    if alpha_overlay is not None:
        for c in range(3):  # Mezcla cada canal (B, G, R)
            roi[:, :, c] = (alpha_overlay * overlay[:, :, c] +
                            (1 - alpha_overlay) * roi[:, :, c])
    else:
        # Sin canal alfa, simplemente reemplaza los píxeles
        frame[y:y+overlay_height, x:x+overlay_width] = overlay

# Función para actualizar el índice del GIF y obtener el siguiente frame
def get_next_gif_frame():
    global gif_index
    if gif_index < len(gif_frames):  # Aumenta el índice solo si hay fotogramas disponibles
        frame = cv2.resize(gif_frames[gif_index], (100, 100))
        gif_index += 1
        return frame
    return None  # Devuelve None después de completar el GIF


def change_gif(score):
    global gif_index
    global gif
    global gif_frames
    
    gif_index=0
    # Cargar el archivo GIF usando PIL
    if score==100:
        gif_path = "./assets/perfect.gif"
    elif score==50:
        gif_path = "./assets/good.gif"
    else:
        gif_path = "./assets/fail.gif"  # Ruta al archivo GIF

    gif = Image.open(gif_path)
    # Convertir el GIF en una lista de frames de imágenes
    gif_frames = []
    for frame in ImageSequence.Iterator(gif):
        frame = frame.convert("RGBA")  # Asegurarse de que el GIF tenga un canal alfa
        gif_frames.append(np.array(frame))
        

def calcular_distancia_escala(puntos):
    hombro_izquierdo = puntos[11]
    hombro_derecho = puntos[12]
    return np.linalg.norm(np.array([hombro_izquierdo["x"], hombro_izquierdo["y"]]) -
                          np.array([hombro_derecho["x"], hombro_derecho["y"]]))

def calcular_similitud(puntos1, puntos2, escala1, escala2):
    if not puntos1 or not puntos2 or len(puntos1) != len(puntos2):
        return 0

    distancias = []
    for p1, p2 in zip(puntos1, puntos2):
        p1_escalado = np.array([p1["x"] / escala1, p1["y"] / escala1, p1["z"] / escala1])
        p2_escalado = np.array([p2["x"] / escala2, p2["y"] / escala2, p2["z"] / escala2])
        distancias.append(np.linalg.norm(p1_escalado - p2_escalado))

    return np.mean(distancias)

def obtener_puntos_por_frame(json_data, frame_buscado):
    for entrada in json_data:
        if entrada["frame"] == frame_buscado:
            return entrada["puntos"]
    return None

def puntuacion(json_file, frame_buscado, results_cam):
    puntos_referencia = obtener_puntos_por_frame(json_file, frame_buscado)
    if puntos_referencia:
        puntos_usuario = [
            {"x": lm.x, "y": lm.y, "z": lm.z, "visibility": lm.visibility}
            for lm in results_cam.pose_landmarks.landmark
        ]

        escala_referencia = calcular_distancia_escala(puntos_referencia)
        escala_usuario = calcular_distancia_escala(puntos_usuario)

        similitud = calcular_similitud(puntos_referencia, puntos_usuario, escala_referencia, escala_usuario)

        print(similitud)

        if similitud < 3:
            change_gif(100)
            return 100
        elif similitud < 5:
            change_gif(50)
            return 50
        else:
            change_gif(0)
            return 0
    return 0

def process_video(cap_video, video_queue, video_frame_rate, frame_width, frame_height):
    global stop_threads
    while cap_video.isOpened() and not stop_threads:
        current_time = pygame.mixer.music.get_pos() / 1_000
        current_frame_index = int(current_time * video_frame_rate)

        cap_video.set(cv2.CAP_PROP_POS_FRAMES, current_frame_index)
        ret_video, frame_video = cap_video.read()
        if not ret_video:
            break

        frame_rgb = cv2.cvtColor(frame_video, cv2.COLOR_BGR2RGB)
        frame_rgb = cv2.resize(frame_rgb, (frame_width, frame_height))  # Reducir tamaño

        gif_frame = get_next_gif_frame()
        if gif_frame is not None:
            overlay_image(frame_rgb, gif_frame, 0, 0)

        video_queue.put(frame_rgb)

    cap_video.release()

def process_camera(cap_camera, camera_queue, json_file, video_frame_rate, frame_width, frame_height):
    global stop_threads
    frame_count = 0
    score = 0
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap_camera.isOpened() and not stop_threads:
            ret_camera, frame_camera = cap_camera.read()
            if not ret_camera:
                break

            frame_camera = cv2.flip(frame_camera, 1)
            frame_camera = cv2.resize(frame_camera, (frame_width, frame_height))
            rgb_frame_cam = cv2.cvtColor(frame_camera, cv2.COLOR_BGR2RGB)

            frame_count += 1
            if frame_count % process_every_n_frames == 0:
                results_cam = pose.process(rgb_frame_cam)
                if results_cam.pose_landmarks:
                    current_time = pygame.mixer.music.get_pos() / 1_000
                    current_frame_index = int(current_time * video_frame_rate)
                    score += puntuacion(json_file, current_frame_index, results_cam)

                    mp_drawing.draw_landmarks(
                        frame_camera, results_cam.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2, circle_radius=2),
                        mp_drawing.DrawingSpec(color=(255, 0, 255), thickness=2, circle_radius=2)
                    )

            cv2.putText(frame_camera, f'Puntuacion: {score}', (10, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            camera_queue.put(frame_camera)

    cap_camera.release()

def jugar_con_puntos(json_file, video_path):
    global stop_threads

    with open(json_file, 'r') as f:
        puntos_video_referencia = json.load(f)

    pygame.init()
    audio_file = 'audio_extraido.mp3'
    pygame.mixer.music.load(audio_file)
    pygame.mixer.music.play()

    cap_video = cv2.VideoCapture(video_path)
    cap_camera = cv2.VideoCapture(0)

    if not cap_video.isOpened() or not cap_camera.isOpened():
        print("Error: No se pudo abrir el video o la cámara.")
        return

    frame_width = int(cap_video.get(cv2.CAP_PROP_FRAME_WIDTH)) // 2  # Reducir a la mitad
    frame_height = int(cap_video.get(cv2.CAP_PROP_FRAME_HEIGHT)) // 2  # Reducir a la mitad
    video_frame_rate = cap_video.get(cv2.CAP_PROP_FPS)

    screen_width = frame_width * 2  # Doblar para mostrar video + cámara
    screen_height = frame_height
    screen = pygame.display.set_mode((screen_width, screen_height))  # Usar dimensiones reducidas

    video_queue = Queue()
    camera_queue = Queue()

    video_thread = Thread(target=process_video, args=(cap_video, video_queue, video_frame_rate, frame_width, frame_height))
    camera_thread = Thread(target=process_camera, args=(cap_camera, camera_queue, puntos_video_referencia, video_frame_rate, frame_width, frame_height))

    video_thread.start()
    camera_thread.start()

    while video_thread.is_alive() or camera_thread.is_alive():
        if not video_queue.empty():
            frame_rgb = video_queue.get()
            current_image = pygame.image.frombuffer(frame_rgb.tobytes(), frame_rgb.shape[1::-1], "RGB")
            screen.blit(current_image, (0, 0))

        if not camera_queue.empty():
            frame_camera = camera_queue.get()
            frame_camera_rgb = cv2.cvtColor(frame_camera, cv2.COLOR_BGR2RGB)
            frame_camera_surface = pygame.image.frombuffer(frame_camera_rgb.tobytes(), frame_camera_rgb.shape[1::-1], "RGB")
            screen.blit(frame_camera_surface, (frame_width, 0))

        pygame.display.flip()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                stop_threads = True
                video_thread.join()
                camera_thread.join()
                cap_video.release()
                cap_camera.release()
                pygame.quit()
                return

    stop_threads = True
    video_thread.join()
    camera_thread.join()
    cap_video.release()
    cap_camera.release()
    pygame.quit()


if __name__ == "__main__":
    modo = input("Seleccione modo: 'extraer' para guardar puntos o 'jugar' para comparar: ")

    if modo == 'extraer':
        extraer_y_guardar_puntos(dance_video, 'puntos_referencia.json')
    elif modo == 'jugar':
        jugar_con_puntos('puntos_referencia.json', dance_video)


18.113399383303076
10.757642716612661
49.41278613508071
9.146528536879464
11.486701177843711
23.017846363749566
6.175157117679301
45.48207176056337
8.894321888841421
8.725809166826988
3.9957589999824865
4.411978531524868
5.206003225302198
3.920628186651439
4.25183353517686
4.332453992293823
4.910661762570328
3.1651554728044777
4.618697002084217
3.85496276713027
4.006228644722784
4.571261829984873
9.436003863333298
6.457013534036639
7.646472940305514
5.574945951445029
7.611838280347926
5.446369463699101
5.481292776150678
5.189663259393166
7.377489453081635
6.961062593325984
5.457822188170198
4.793495058696249
3.999291336414404
6.045808333526707
5.167473875159979
4.479851846436889
5.017840552457486
6.806062013260599
6.823157200362158
5.827494026162199
5.89671878826592
4.6650101515070785
5.474006398933915
5.810630648322229
5.567091015382705
5.632912478407797
6.484640351412736
8.22203691053085
5.702990248168372
5.760297522363961
6.919116315709512
6.2598967259188365
6.480956060929415
5.7202